In [1]:
import pandas as pd

In [2]:
def df_info(df):
    
    '''
    Функция выводит наиболее важную информацию о данных в компактном представлении
    '''
    
    print('Состав:', df.head(), sep='\n')
    print(df.info(), sep='\n')
    print('Размер:', df.shape, sep='\n')
    print('Дубликаты:', sum(df.duplicated()), sep='\n')
    pass

In [3]:
clust = pd.read_csv('address_clust.csv')
df_info(clust)

Состав:
   address_id  cluster_id
0    71413451           1
1    71411914           1
2    71410369           1
3    71410100           1
4    71410040           1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6996 entries, 0 to 6995
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   address_id  6996 non-null   int64
 1   cluster_id  6996 non-null   int64
dtypes: int64(2)
memory usage: 109.4 KB
None
Размер:
(6996, 2)
Дубликаты:
0


In [4]:
stats = pd.read_csv('address_stats.csv')
df_info(stats)

Состав:
         id  address_id  transaction_id   received       sent
0  87134765    46402336        19162323        0.0  1800000.0
1  87134766    45919235        19162323        0.0  1071052.0
2  87134767    46529090        19162323  1000000.0        0.0
3  87134768    46529091        19162323  1821052.0        0.0
4  87154706    46529090        19166856        0.0  1000000.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76724 entries, 0 to 76723
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              76724 non-null  int64  
 1   address_id      76724 non-null  int64  
 2   transaction_id  76724 non-null  int64  
 3   received        76724 non-null  float64
 4   sent            76724 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 2.9 MB
None
Размер:
(76724, 5)
Дубликаты:
22279


In [5]:
stats.drop_duplicates(inplace=True)

In [6]:
full = stats.merge(clust, on='address_id', how='left')
df_info(full)

Состав:
         id  address_id  transaction_id   received       sent  cluster_id
0  87134765    46402336        19162323        0.0  1800000.0         NaN
1  87134766    45919235        19162323        0.0  1071052.0         NaN
2  87134767    46529090        19162323  1000000.0        0.0         1.0
3  87134768    46529091        19162323  1821052.0        0.0         NaN
4  87154706    46529090        19166856        0.0  1000000.0         1.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 54445 entries, 0 to 54444
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              54445 non-null  int64  
 1   address_id      54445 non-null  int64  
 2   transaction_id  54445 non-null  int64  
 3   received        54445 non-null  float64
 4   sent            54445 non-null  float64
 5   cluster_id      37625 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 2.9 MB
None
Размер:
(54445, 6)
Дублик

In [7]:
full.fillna(0, inplace=True)
del clust, stats

пару вопросов...

In [8]:
#Может ли в одной тразакции быть несколько адресов-отправителей?

gr = full[full.sent>0].groupby('transaction_id')['address_id'].nunique()

if sum(gr[gr>1]):
    print('Да')
else:
    print('Нет')

Да


In [9]:
#Могут ли в одной тразакции быть разные кластеры-отправители?

gr = full[full.sent>0].groupby('transaction_id')['cluster_id'].nunique()

if sum(gr[gr>1]):
    print('Да')
else:
    print('Нет')

Нет


In [10]:
#Что насчет кластеров-получателей?

gr = full[full.received>0].groupby('transaction_id')['cluster_id'].nunique()

if sum(gr[gr>1]):
    print('Да')
else:
    print('Нет')

Да


Сколько биткоинов отправил кластер 1 в кластер 2 и сколько кластер 2 отправил в кластер 1?

In [11]:
def answ(func):
    def wrapper():
        return(f'{func(sender, receiver)} btc отправил кластер {sender} в кластер {receiver}')
    return wrapper

In [12]:
@answ
def how_many(sender, receiver):
    '''
    Т. к. в одной транзакции получателями могут выступать как участники кластеров, 
    так и внешние получатели одновременно
    то сумму отправленных btc с кластера 'sender' будем считать по количеству 
    полученных другим кластером 'receiver' монет
    '''
    sen_con = (full.cluster_id==sender)&(full.sent>0)
    rec_con = (full.cluster_id==receiver)&(full.received>0)
    
    sent_id = full.loc[sen_con, 'transaction_id']
    sent_sum = full.loc[rec_con&full.transaction_id.isin(sent_id), 'received'].sum()
    
    return sent_sum/10**8

In [13]:
sender=1 
receiver=2
how_many()

'64.62162 btc отправил кластер 1 в кластер 2'

In [14]:
sender=2
receiver=1
how_many()

'67.91 btc отправил кластер 2 в кластер 1'

In [15]:
#Сколько биткоинов отправил кластер 1 в кластер 0
sender=1
receiver=0
how_many()

'291.5108346 btc отправил кластер 1 в кластер 0'

In [16]:
#и сколько кластер 0 отправил в кластер 1?
sender=0
receiver=1
how_many()

'288.7905886 btc отправил кластер 0 в кластер 1'

In [17]:
#Сколько биткоинов отправил кластер 2 в кластер 0
sender=2
receiver=0
how_many()

'2.43507 btc отправил кластер 2 в кластер 0'

In [18]:
#и сколько кластер 0 отправил в кластер 2?
sender=0
receiver=2
how_many()

'6.09640861 btc отправил кластер 0 в кластер 2'

In [19]:
#Сколько биткоинов потратил кластер 1 и кластер 2 на fee?

def clfee(sender):
    
    sen_id = full.loc[(full.cluster_id==sender)&(full.sent>0), 'transaction_id']
    sent_sum = full.loc[full.transaction_id.isin(sen_id), 'sent'].sum()
    rec_sum = full.loc[full.transaction_id.isin(sen_id), 'received'].sum()
    fee = (sent_sum-rec_sum)/10**8
    return f'Кластер {sender}, потратил {fee} btc на fee'

In [20]:
clfee(1)

'Кластер 1, потратил 0.502344 btc на fee'

In [21]:
clfee(2)

'Кластер 2, потратил 0.37295861 btc на fee'